# Train Emotion Recognition
ResMaskingNet which is available in PyFeat uses 142.9 million parameters with 74% accuracy.
Human emotion detection only reaches 65+-5%, poor golden standard

3-15 million

In [48]:
import wandb
import random
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [ ]:
WANDB_API_KEY = "9ff9cdd1ffada991cc4f457217b7dfa6d0d7ce3b"
WANDB_NOTEBOOK_NAME = "Train Emotion Recognition"
wandb.login()

In [ ]:
wandb.init(
    project = "Furhat Coach",

    config = {
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    }
)

epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    wandb.log({"acc": acc, "loss": loss})

wandb.finish()

In [18]:
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(70, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 7),
        )
    
    def forward(self, inputs):
        return self.net(inputs)

In [44]:
class MultiEmoVA(Dataset):
    def __init__(self, data_path) -> None:
        super().__init__()
        data=pd.read_csv(data_path)
        self.inputs = torch.tensor(data.drop(["emotion","path", "index"], axis=1).to_numpy(np.float32))
        self.index2label = [label for label in data["emotion"].unique()]
        label2index = {label: i for i, label in enumerate(self.index2label)}
        self.labels = torch.tensor(data["emotion"].apply(lambda x: label2index[x]))

    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

    @staticmethod
    def label_dict():
        data=pd.read_csv("../dataset/dataset.csv")
        return [label for label in data["emotion"].unique()]

In [ ]:
def hyper_model(l1, learn):
    device = "cuda"
    data = MultiEmoVA("../dataset/dataset.csv")
    unlabeled = MultiEmoVA("../dataset/test_to_submit.csv")

    train, val, test = random_split(data, [0.7, 0.2, 0.1])

    model = MyNeuralNetwork(l1)


    train_loader = DataLoader(train)
    val_loader = DataLoader(val)
    test_loader = DataLoader(test)

    loss_fn = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr = learn)

    n_epochs = 20

    for epoch in range(n_epochs):
        for inputs, labels in train_loader:
            output = model(inputs)
            loss = loss_fn(output, labels)
            loss.backward()
            optim.step()
            optim.zero_grad()

        correct = 0
        for inputs, labels in val_loader:
            output = model(inputs)
            correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
        #print(f"Validation accuracy is: {correct/len(val)}")


    correct = 0
    for inputs, labels, in test_loader:
        output = model(inputs)
        correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
    print(f"Hidden nodes: {l1}, Learning rate {learn}")
    print(f"Test accuracy is: {correct/len(test)}")


    unlabeled_loader = DataLoader(unlabeled)
    out = pd.DataFrame(list(map(lambda x: MultiEmoVA.label_dict()[model(x).softmax(dim=1).argmax(dim=1)], unlabeled_loader)))
    out.to_csv('../processed/outputs', index = False, header = False)


In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [45]:
model = MyNeuralNetwork().to(device)

def init_weights(layer):
    if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight)

model.net.apply(init_weights)

loss_func = nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr = 0.001)

n_epochs = 100


# use nn.Softmax() before prediction


In [51]:
data = MultiEmoVA("data\DiffusionFER\DiffusionEmotion_S\cropped\labels.csv")
train, test = train_test_split(data, test_size = 0.3, stratify = data.labels


NameError: name 'emotion' is not defined

In [ ]:
for epoch in range(n_epochs):
    for inputs, labels in train_loader:
        output = model(inputs)
        loss = loss_func(output, labels)
        loss.backward()
        optim.step()
        optim.zero_grad()

    correct = 0
    for inputs, labels in val_loader:
        output = model(inputs)
        correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()